This is Assignment 1. Deadline for submission is October, 3, 11pm EST. Please send your solutions in a gzipped archive to dmm2017@med.cornell.edu, with copy to chb4004@med.cornell.edu. No extensions will be granted.

Your archive should contain this ipynb file with the code you wrote. However, if you are not comfortable with python, you can write in any other programming language, but provide instructions how to run/compile your code. 

Assignment 1, Aditi Gopalan

Task 1:
====

Your task is to find the position of an insertion in a given genome. The length of each insertion is unknown. You will be given five input files. Random sequence contain A,C,G,T characters and probabilities of each character are equal.

01.fasta - a real genomic sequence with a random sequence inserted in a random place.  

02.fasta - a heavily mutated real genomic sequence with a random sequence inserted in a random place

03.fasta - a real genomic sequence with real genomic sequence from a completely different organism inserted in a random place 

04.fasta - a real genomic sequence with real genomic sequence from a completely different organism inserted in a random place

05.fasta - a randomly generated sequence with a real genomic sequence inserted in a random place

**Output format**

For each input file output two positions - start and end position of inserted sequence.

**Grading**

Your solution will be graded based on a secret formula that penalizes the solution if your output start and end positions are far away from the truth start and end.
Only answers will be graded, but you will have to provide code that you used to get this solution.


 **OUTPUT:**

I was able to identify areas of insertion on the gene: 

fasta1: 17800 22000

fasta2: 1490250 1563700 

fasta3: 4900 6380

fasta4: 362870 410000

fasta5: 1177000 1235123 

In [ ]:
import pandas as pd
from Bio import SeqIO
import operator
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

'''
Collaborated with Max Chao 

My approach is to check the GC content on each fasta file because I assume the insertions will show a different pattern of GC content. This will help to identify regions of insertion. 

For fasta files 2 and 5 I determined the kmer position in order to narrow down the region of insertion. 

OUTPUT:
I was able to identify areas of insertion on the gene:
fasta1: 17800 22000
fasta2: 1490250 1563700 
fasta3: 4900 6380
fasta4: 362870 410000
fasta5: 1177000 1235123

'''
def get_gc(seq): #counts GC content
	count_c = 0
	count_g = 0
	for i in seq:
		if i == "C":
			count_c +=  1
		if i == "G":
			count_g += 1
	total = count_c + count_g
	return total


fasta_sequences = SeqIO.parse(open("/mnt/c/ds/01.fasta"),'fasta') #parsing a fasta file
with open("/mnt/c/ds/01.fasta") as fasta_file:
	for fasta in fasta_sequences:
		name, seq = fasta.id, str(fasta.seq)
		print("Len", len(seq))
		bin_size = len(seq)//100 #Splitting the sequence into 100 bins of equal nucleotides
		CG_dict = {} #Dictionary that stores the GC composition of each bin
		for n in range(0,100):
			if n == 0:
				bin_n = seq[0:bin_size]
				gc = get_gc(bin_n)

			elif n == bin_size-1:
				bin_n = seq[(bin_size*n):]
				gc = get_gc(bin_n)

			else:
				bin_n = seq[bin_size*n:bin_size*(n+1)]
		    gc = get_gc(bin_n)
			gc_comp = gc/len(bin_n)
			CG_dict[n] = gc_comp
		plt.bar(*zip(*CG_dict.items())) #Plotting GC composition to identify areas of irregularity
		plt.savefig('fasta1.png')


		insert_pos = []
		for k, v in CG_dict.items():
			if v> 0.48:#Setting threshold value from region of irregularity in the graph
				insert_pos.append(k)
		start = min(insert_pos)
		end = max(insert_pos)
		print(start, end)
		s = start*bin_size
		e = end*bin_size
		print("The insertion is approximately between", (s,e) )

		
'''
Since we cannot tell the areas of insertion from just GC composition for fasta2 and fasta5, I decided to try a kmer approach to identify these regions
using kmers, for fast 2 and fast 5

Note that this comment section separates two differing python files of code
'''
import pandas as pd
from Bio import SeqIO
import operator
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

def find_kmer(name, seq): #function counts kmers in a given sequence
	print("%s\n################################" %name)
	kmers = {} #this is a dictionary that will record kmers and their frequencies
	pos = {} #this is a dictionary of all the kmer positions
	k = 7 #this is kmer size
	for i in range(len(seq) - k + 1): #counting kmers
		kmer = seq[i:i+k]
		pos_k = [i, i+k]
		if kmer in kmers:
			kmers[kmer] += 1
		else:
			kmers[kmer] = 1
		if pos_k in pos:
			pos[kmer] += 1
		else:
			pos[kmer] = 1
	data_items = kmers.items()
	data_list = list(data_items)
	df = pd.DataFrame(data_list)
	print(df)

	data_items1 = pos.items()
	data_list1 = list(data_items1)
	df1 = pd.DataFrame(data_list1)
	print("df1", df1)

	insert_pos = []
	for k, v in kmers.items():
		if  v <= 10:
			for k2, v2 in pos.items():
				   c2 = pos['k']
			       insert_pos.append(c2)

	start = min(insert_pos)
	end = max(insert_pos)
	print("The insertion is approximately between", (start,end) )

fasta_sequences = SeqIO.parse(open("/mnt/c/ds/02.fasta"),'fasta') #parsing a fasta file
with open("/mnt/c/ds/02.fasta") as fasta_file:
	for fasta in fasta_sequences:
		name, sequence = fasta.id, str(fasta.seq)
		find_kmer(name, sequence)


Task 2:
====
Suppose that you identified the outbreak of unknown virus in a distant village. Villagers want to know who was the first person, who get the virus. Unfortunately, virus has a long and variable incubation period and it can transmit between people even during incubation. You sampled and sequenced 100 randomly selected patients from the village and assembled viral RNA. Your task is to order these people according to their infection time.

When an infected person touches a healthy person, he/she gets the virus, during transmission virus mutates but it's DNA doesn't change afterwards. It can get some SNPs (single nucleotide polymorphisms) and indels during the mutation procedure. In our simulations we uniformly and randomly introduce 0-3 deletions and 1-5 insertions per transmission.

Your solution will be rated according to the number of correct strictly ordered pairs in our simulations. We simulated data according to some rooted tree with zero patient being in the root. It means that there is no reinfection.

You should get your answer in some kind of automated way. Please, provide your code for computing global alignment and final arrangement. Briefly describe your ideas in a write-up or separate notebook cell.

**Input format:**

Input file contains 100 DNA sequences, one per line. In i-th line there is a assembled virus from i-th patient. 

**Output format:**

Please provide your answer in a separate file. Your file should contain 100 distinct integer numbers, one per line. Number $n$ on i-th line means that you think that n-th person was the i-th person to get the virus.
  
Answer:  

2  

3  

1  

means that transmission history is [2->3, 2->1] or [2->3->1] 

Input file can be found on the Google Disk.

**Score distribution for this task:**

1. 50% - correct ordering
2. 30% - code quality
3. 20% - idea correctness

In [ ]:
import itertools
import numpy as np
import pandas as pd
import itertools
import regex as re
import os
import glob
import pandas as pd
os.chdir("/mnt/c/ds")

"""
Collaborated with Yohyoh Wang and Phoebe Fei 

I first sorted the input sequences based on length into five bins of 20 sequences each. I performed pairwise global alignment on each of these bins to compute the alignments and score.
Once all the bins were aligned, I counted the number of mutatational events. If consecutive, I counted them as one event. Then, I sorted pairs based on ascending order of scores to obtain the order of sequences.
The sequences with the maximum score were the most mutated or "newest" sequences
"""

def aligner(s1, s2):
	#scores
	mismatch = 1
	match = 0
	indel = 1

	#Initializing the matrix
	for a in range(1, m):
		mat[a, 0] = a
		trace[a,0] = 1
	for b in range(1, n):
		mat[0, b] = b
		trace[0, b] = 2

	#Scoring matrix based on match/mismatch
	for i in range(1, m):
		for j in range(1, n):

			if seq1[i-1] == seq2[j-1]:
				m_score = mat[i-1, j-1] + match
			else:
				m_score = mat[i-1, j-1] + mismatch

			choices = [mat[i-1, j] + 1, mat[i, j-1] + 1, m_score]
			mat[i,j] = min(choices)
			#Traceback matrix
			trace[i, j] = choices.index(min(choices)) 

	#Traceback matrix
	pos0 = [seq1[-1]]
	pos1 = [seq2[-1]]
	i = -1 # index for pos0
	j = -1 # index for pos1

	while (-i <= len(seq1)-1) & (-j <= len(seq2)-1):
		if trace[i, j] == 0:
			pos0.append('-')
			pos1.append(seq2[j])
			i -= 1
		elif trace[i, j] == 1:
			pos0.append(seq1[i])
			pos1.append('-')
			j -= 1
		elif trace[i, j] == 2:
			pos0.append(seq1[i-1])
			pos1.append(seq2[j-1])
			i -= 1
			j -=1

	pos0 = ''.join(pos0[::-1])
	pos1 = ''.join(pos1[::-1])

	print(pos0,pos1)

	#count number of insertions or deletions
	def count_events(p):
		res = re.finditer('-', p) #find locations with the gap
		res_start = [i.start() for i in res]

		i = 0
		events = 0

		while i < len(res_start):
			if i == 0: #if first nucleotide change
				events += 1
			else:
				if res_start[i-1]+1 != res_start[i]: #if not consecutive
					events += 1
			i+=1
		return events

	e0 = count_events(pos0)
	e1 = count_events(pos1)
	return e0, e1

#Parsing the sequence file
with open('/mnt/c/ds/input_data.txt') as f:
	seq_list = f.readlines()
#seq_list = ['ATGC', 'ATGCGACCCC', 'ATGGGTATAT', 'TATATGATGC', 'GGGGCCCTCTATGCC', 'ATGTCTCTCCCC', 'ATTATATCTCTCGC', 'ATGCTCTCTCC', 'ATGTCCTCCC', 'GTGTGATGC', 'ATGCTCAGCTACGC', 'ATGCTCGACC']
#seq_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
seq_list.sort(key=len)

bin_1 = seq_list[:20]
bin_2 = seq_list[20:40]
bin_3 = seq_list[40:60]
bin_4 = seq_list[60:80]
bin_5 = seq_list[80:]

#seq_list = ['ATGC', 'ATGCC']
#pair = []
final_comb = []
#pair_events_sum = []

#Iterating throgh all bins
for k in range(1,6):
	print(k)
	binname = locals()["bin_"+str(k)]
	print("this is binname", str(k))
	for r in range(len(binname) + 1):
		combinations_object = itertools.combinations(binname, 2)
		combinations_list = list(combinations_object)
		final_comb += combinations_list
		myset = set(final_comb)
		final_comb = list(myset)

	final_comb_1 = [list(ele) for ele in final_comb]
	#print("Final comb", final_comb_1)


	pair = []
	pair_events_sum = []
#Finding combinations of 2 in all bins and aligning them
	for i in final_comb:
		#print("this is i", i)
		for seq1, seq2 in zip(i[::2], i[1::2]):
			m = len(seq1) + 1 #seq1 length
			n = len(seq2) + 1 #seq2 length
			mat = np.zeros((m,n)) #scoring matrix of dimension mxn
			trace = np.zeros((m,n)) #traceback matrix of dimension mxn
			events0, events1 = aligner(seq1, seq2)
			sum = events0 + events1
			#print("Sum", sum)
			pair_events_sum.append(sum)
			#print("This is pair_events_sum_list", pair_events_sum)
			pair.append(i)
			#print("This is pair and sum of events", pair, pair_events_sum)
	print("finished aligning bin", str(k))
	scores_df_bin = pd.DataFrame({'pair':pair, 'score':pair_events_sum})
	scores_df_bin.to_csv("Score for bin_"+ str(k) + ".csv", index = False)

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
list_of_pairs = combined_csv['pair'].tolist()

combined_csv.to_csv( "Final score for all bins.csv", index=False)

Please note that this cell separates 2 different python files that I used for task 2

In [ ]:
import pandas as pd
import ast

'''
This code maps sequences to their number in order to output a file that has number n
on i-th line, which means that n-th person was the i-th person to get the virus.
'''
def MapWordsToPos(dict_1,pair_list):
    new_pair_list = []
    for pair in pairlist:
        new_l = []
        #print("This is pair", pair)
        for seq in pair:
            #print("This is seq", seq)
            if seq in seq_dict.keys():
                val = seq_dict.get(seq)
                #print("This is seq val", seq,val)
                new_l.append(val)
                print("This is new list", new_l)
        new_pair_list.append(new_l)
        print("This is new pair list", new_pair_list)
    return new_pair_list

'''
pairlist = [('a','b'), ('c', 'd'), ('e', 'f'), ('b', 'c')]
seq_list = ['a', 'b', 'c', 'd', 'e', 'f']
score = [1,3,2,4]
'''

df = pd.read_csv("/mnt/c/ds/Final score for all bins.csv", converters={'pair': ast.literal_eval})
print(df)
pairlist = df['pair'].to_list()
score = df['score'].to_list()
#print("pairlist", pairlist)
#print("score", score)

with open('/mnt/c/ds/input_data.txt') as f:
    seq_list = [i.strip('\n') for i in f]
print(seq_list)
vals = list(range(1,101))
seq_dict = {}

for key in seq_list:
    for value in vals:
        seq_dict[key] = value
        vals.remove(value)
        break
#print("Dict", seq_dict)
#print("Pair", pairlist[0])


new_pair_list = MapWordsToPos(seq_dict,seq_list)
print(new_pair_list)


d = {'Pair Indices':new_pair_list,'Score':score}
df2 = pd.DataFrame(d)
df2 = df2.sort_values(by=['Score'])
#print(df2)

pair_list = df2['Pair Indices'].to_list()
order_list = []
order_list.extend(pair_list[0])
for pair in pair_list:
    for seq in pair:
        if seq in order_list:
            continue
        else:
            order_list.append(seq)
#print(order_list)


textfile = open("task2_final_output.txt", "w")
for element in order_list:
    textfile.write(str(element) + "\n")
textfile.close()

The order is as below, I have attached the output file in the email

36
 34
 68
 83
 26
 6
 16
 56
 88
 50
 40
 51
 43
 86
 77
 13
 70
 25
 53
 49
 22
 8
 15
 41
 58
 93
 4
 69
 63
 44
 82
 30
 39
 46
 12
 80
 59
 92
 37
 95
 90
 84
 47
 76
 99
 54
 73
 61
 9
 67
 57
 3
 28
 71
 87
 100
 78
 14
 27
 24
 74
 65
 75
 45
 7
 23
 48
 17
 38
 98
 96
 97
 20
 64
 21
 60
 52
 33
 29
 2
 19
 55
 79
 62
 10
 31
 85
 89
 5
 72
 18
 91
 1
 66
 32
 35
 11
 94
 81
 42